In [ ]:
import pandas as pd
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import gradio as gr
import os

In [11]:
file_path = "/Users/priyal/Documents/honours/extracted_papers_info.csv"
data = pd.read_csv(file_path)

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

documents = []
for idx, row in data.iterrows():
    title = row.get("Title", "")
    abstract = row.get("Abstract", "")
    author = row.get("Author Names", "Unknown")

    if isinstance(abstract, str) and abstract.strip():
        chunks = text_splitter.split_text(abstract)

        for chunk in chunks:
            documents.append(
                Document(
                    page_content=chunk,
                    metadata={
                        "citation": f"{title} (Citation ID: {idx})",
                        "author": author,
                    },
                )
            )

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

faiss_index = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
retriever = faiss_index.as_retriever()

In [14]:
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.7)

In [15]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/4y/9y5ld1xn3zn_rhkhdmmr63zr0000gn/T/ipykernel_17332/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [52]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True, 
)

In [75]:
def chat_with_rag(user_input, chat_history=[]):
    history = memory.chat_memory.messages

    formatted_history = "\n".join(
        [
            (
                f"User: {msg.content}"
                if isinstance(msg, HumanMessage)
                else f"Bot: {msg.content}"
            )
            for msg in history[-5:]
        ]
    )

    full_query = f"Here is our conversation so far:\n{formatted_history}\n\nNow answer this: {user_input}"

    result = rag_chain.invoke({"query": full_query})

    response = result.get("result", "Sorry, I couldn't generate a response.")

    source_docs = result.get("source_documents", [])
    citations = [
        f"- {doc.metadata.get('citation', 'Unknown Source')}" for doc in source_docs
    ]
    citations_str = "\n".join(citations) if citations else "No citations available."

    memory.save_context({"input": user_input}, {"output": response})

    chat_history.append((user_input, f"{response}\n\n**Citations:**\n{citations_str}"))

    return chat_history  

In [79]:
with gr.Blocks() as demo:
    gr.Markdown("# Multiturn testing")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(
        label="Type your question here...", placeholder="Ask about Ayurveda..."
    )
    submit_btn = gr.Button("Send")

    submit_btn.click(chat_with_rag, inputs=[user_input, chatbot], outputs=[chatbot])
    user_input.submit(chat_with_rag, inputs=[user_input, chatbot], outputs=[chatbot])

/Users/priyal/.pyenv/versions/3.10.4/lib/python3.10/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [80]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [66]:
# print("Chatbot is ready! Type 'exit' to stop.\n")

# chat_history = []
# while True:
#     user_query = input("You: ")
#     if user_query.lower() == "exit":
#         break

#     bot_response = chat_with_rag(user_query)
#     print(f"Bot: {bot_response}\n")

Chatbot is ready! Type 'exit' to stop.

DEBUG - Chat History After Update:
 [HumanMessage(content='How does Ayurveda contribute to overall health, and what are its main principles as described in studies?', additional_kwargs={}, response_metadata={}), AIMessage(content=' Ayurveda contributes to overall health by promoting a state of complete physical, mental, and spiritual well-being. Its main principles include a focus on predictive, preventive, and personalized medicine, personalized counseling on lifestyle measures, and an emphasis on self-awareness and good relationships with others and nature. It also incorporates factors such as health education, individual awareness, and spirituality in healthcare to improve overall well-being and promote healthy aging.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how does it incorporate spirituality?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The paper mentions that the Vedic approach to therapy i